<a href="https://colab.research.google.com/github/Razzf/MachineLearningTests/blob/master/KMclustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import sklearn
from sklearn.preprocessing import scale
from sklearn.datasets import load_digits
from sklearn.cluster import KMeans
from sklearn import metrics

In [0]:
digits = load_digits()
data = scale(digits.data)
y = digits.target

k = 10
samples, features = data.shape

In [0]:
def bench_k_means(estimator, name, data):
    estimator.fit(data)
    print('%-9s\t%i\t%.3f\t%.3f\t%.3f\t%.3f\t%.3f\t%.3f'
          % (name, estimator.inertia_,
             metrics.homogeneity_score(y, estimator.labels_),
             metrics.completeness_score(y, estimator.labels_),
             metrics.v_measure_score(y, estimator.labels_),
             metrics.adjusted_rand_score(y, estimator.labels_),
             metrics.adjusted_mutual_info_score(y,  estimator.labels_),
             metrics.silhouette_score(data, estimator.labels_,
                                      metric='euclidean')))

In [5]:
clf = KMeans(n_clusters=k, init="random", n_init=10)
bench_k_means(clf, "1", data)

1        	69407	0.601	0.649	0.624	0.466	0.620	0.146
